# IMDB 영화평 감성분석

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../00.data/IMDB/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [4]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [5]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />', ' ')

In [6]:
# 영어 이외의 문자는 공백으로 변환(숫자 등)
# 앞에 들어가는 ^는 not의 의미 -> a-z, A-Z가 '아닌 놈들은' 공백으로 바꿔라
import re

df['review'] = df.review.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

In [8]:
from sklearn.model_selection import train_test_split

feature_df = df.drop(['id', 'sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size =0.3, random_state=156
)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- CountVectorizer

In [13]:
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))
count_vect.fit(X_train.review)
X_train_count = count_vect.transform(X_train.review)
X_test_count = count_vect.transform(X_test.review)

In [14]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_count, y_train)
pred = lr_clf.predict(X_test_count)
accuracy_score(y_test, pred)

0.886

- TfidfVectorizer

In [25]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect.fit(X_train.review)
X_train_tfidf = tfidf_vect.transform(X_train.review)
X_test_tfidf = tfidf_vect.transform(X_test.review)

In [26]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf, y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.8936

### Model 저장하고 불러오기

In [27]:
import joblib
joblib.dump(tfidf_vect, 'model/imdb_vect.pkl')
joblib.dump(lr_clf, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [28]:
# 변수지우기
del tfidf_vect
del lr_clf

In [29]:
new_vect = joblib.load('model/imdb_vect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [30]:
new_X_test = new_vect.transform(X_test.review)

In [31]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8936

- 모델을 저장하고 불러오기로 사용하면, 이후에 모델을 재사용 할 때에 보다 시간을 save 할 수 있다.
- web 프로그램을 만드는경우, 이미 만들어놓은 파일을 디스크로 만들어 화면에 표시가하고, 그에 따라 훈련시킨 모델로 긍정/부정의 평가를 내리면 된다.
- 그렇기때문에, save-load의 과정은 꼭 갖고가는것이 좋다.

### Pipeline을 써서 학습/예측/평가

In [33]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))), ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logistic Regression 정확도 : {acc:.4f}')

Count Vectorizer + Logistic Regression 정확도 : 0.8860


In [34]:
joblib.dump(pipeline, 'model/pipline.pkl')

['model/pipline.pkl']

In [35]:
new_pipe = joblib.load('model/pipline.pkl')

In [36]:
pred = new_pipe.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logistic Regression 정확도 : {acc:.4f}')

Count Vectorizer + Logistic Regression 정확도 : 0.8860


In [41]:
from sklearn.model_selection import GridSearchCV

params = {
    'count_vect__max_df': [300, 700],
    'lr_clf__C': [1, 10]
}

grid_pipe = GridSearchCV(new_pipe, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  4.7min finished
{'count_vect__max_df': 700, 'lr_clf__C': 1} 0.8736571714181699


In [43]:
from sklearn.model_selection import GridSearchCV

params = {
    'count_vect__max_df': [500,700,1000],
    'lr_clf__C': [1, 10, 20]
}

grid_pipe = GridSearchCV(new_pipe, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 11.4min finished
{'count_vect__max_df': 1000, 'lr_clf__C': 1} 0.8782860269423692


In [44]:
from sklearn.model_selection import GridSearchCV

params = {
    'count_vect__max_df': [500,700,900],
    'lr_clf__C': [10,20,30]
}
S
grid_pipe = GridSearchCV(new_pipe, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 11.7min finished
{'count_vect__max_df': 900, 'lr_clf__C': 10} 0.8764573901994654
